# This notebook perform simulations of phase transitions in cesium lead iodide (CsPbI3)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ase.optimize.sciopt import *               
from ase.utils.geometry import *
from ase.lattice.spacegroup import crystal
from ase.visualize import view
from ase.lattice.surface import surface
from ase import Atoms
from ase.io import *
from ase.io import read, write
from ase.io.cif import read_cif
from ase.io.vasp import write_vasp
from abtem.visualize import show_atoms
from ase.io.lammpsdata import write_lammps_data
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
import subprocess
import plumed
import nglview

/home/ahlawat/.local/lib/python3.9/site-packages/ase/utils/geometry.py:8: UserWarning: Moved to ase.geometry and ase.build
  warnings.warn('Moved to ase.geometry and ase.build')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/surface.py:17: UserWarning: Moved to ase.build
  warnings.warn('Moved to ase.build')


# ortho-CsPbI3

In [23]:
unit_cell = read('ortho.cif')

rep1 = 2
rep2 = 2
rep3 = 2

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.85
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.83
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 1.64
    i = i + 1

supercell.set_initial_charges(charges=charge_array)

#view(supercell)
print(supercell) 

write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           real
atom_style      full

variable        p_id world   1

variable        temperature equal 300.0
variable        temperature2 equal 300.0
variable        tempDamp equal 100 # ps

variable        pressure equal 1.00
variable        pressureDamp equal 500 #ps

variable        seed world 1428

read_data       data.CPI

mass            1 132.904999 # Cs
mass            2 126.90447  # I
mass            3 204.199997 # Pb

variable        freq equal 500

pair_style      lj/cut/coul/long 10.0 10.0
pair_modify     mix arithmetic
neighbor        0.3 bin
neigh_modify    check yes delay 0
kspace_style    pppm 1e-4
dielectric 1.0

# original parameters
#variable AI     equal "0.042784417"
#variable rI     equal "5.04975"
#variable ACs    equal "0.0898565009" 
#variable rCs    equal "3.521013"
#variable ACsI   equal "14.0467571702"
#variable rCsI   equal "3.39790"
#variable APbI   equal "4.7050"
#variable rPbI   equal "2.800"
#variable APbCs   equal "4.01050"
#variable rPbCs   equal "4.100"

variable APb     equal "0.0051886219"
variable rPb     equal "4.043"
variable AI      equal "0.042784417"
variable rI      equal "5.04975"
variable ACs     equal "0.089856500"
variable rCs     equal "3.521013"

variable ACsI    equal "0.046757170"
variable rCsI    equal "4.408"
variable APbI    equal "0.507295"
variable rPbI    equal "3.103"


pair_coeff      1 1 ${ACs}    ${rCs} 
pair_coeff      2 2 ${AI}     ${rI}  
pair_coeff      3 3 ${APb}    ${rPb}     
pair_coeff      1 2 ${ACsI}   ${rCsI}    
pair_coeff      2 3 ${APbI}   ${rPbI}  

thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
restart         ${freq} restart.0 restart.2

# Minimization

# Minimization
min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-3 1.0e-5 1000 10000
unfix 1
write_data      data.min
reset_timestep  0

# NVT
dump            myDump1 all atom 500 out.0.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature2} ${tempDamp} ${seed}
fix             2 all nve

timestep        2.0

velocity        all create ${temperature} ${seed} dist gaussian
run             10000

unfix           1
unfix           2

write_data      data.NVT

undump          myDump1
reset_timestep  0

# NPT

dump            myDump2 all atom 500 out.ortho.lammpstrj 

fix             1 all temp/csvr ${temperature2} ${temperature2} ${tempDamp} ${seed}
fix             2 all nph tri ${pressure} ${pressure} ${pressureDamp} 
fix             3 all momentum 10000 linear 1 1 1

run             10000

unfix           1
unfix           2
unfix           3

undump          myDump2

reset_timestep  0

write_restart   restart.file
write_data      data.eq

""",file=f)


Atoms(symbols='Cs32I96Pb32', pbc=True, cell=[17.712, 17.152, 24.944], initial_charges=..., spacegroup_kinds=...)


In [24]:
subprocess.run("nohup lmp_intel_cpu_intelmpi < start.lmp &",shell=True)

CompletedProcess(args='nohup lmp_intel_cpu_intelmpi < start.lmp &', returncode=0)

LAMMPS (23 Jun 2022)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (17.712 17.152 24.944) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  160 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.004 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.28323381
  grid = 12 12 15
  stencil order = 5
  estimated absolute RMS force accuracy = 0.015512643
  estimated relative force accuracy = 4.6715864e-05
  using single precision MKL FFT
  3d grid and FFT values/proc = 5780 2160
Generate

      9000   306.44929     -14554.771      145.24127     -14409.53       11128.952      16.357203      16.165949      23.07173       0              0              0            
      9500   243.88129     -14553.264      115.58725     -14437.677      11765.154      16.357203      16.165949      23.07173       0              0              0            
     10000   309.69203     -14549.719      146.77817     -14402.94       11464.526      16.357203      16.165949      23.07173       0              0              0            
Loop time of 4.30049 on 1 procs for 10000 steps with 160 atoms

Performance: 401.815 ns/day, 0.060 hours/ns, 2325.317 timesteps/s
100.0% CPU use with 1 MPI tasks x 1 OpenMP threads

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 1.8921     | 1.8921     | 1.8921     |   0.0 | 44.00
Bond    | 0.00090971 | 0.00090971 | 0.00090971 |   0.0 |  0.02
Kspace

# hexagonal CsPbI3

In [25]:
unit_cell = read('delta.cif')

rep1 = 2
rep2 = 4
rep3 = 1

supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)

# set point charges
i = 0
num_atoms = len(supercell.get_chemical_symbols())
charge_array = [0]*num_atoms
while i < num_atoms:
    if(supercell.get_chemical_symbols()[i] == 'Cs'):
        charge_array[i]= 0.85
    if(supercell.get_chemical_symbols()[i] == 'I'):
        charge_array[i]= -0.83
    if(supercell.get_chemical_symbols()[i] == 'Pb'):
        charge_array[i]= 1.64
    i = i + 1

supercell.set_initial_charges(charges=charge_array)

#view(supercell)
print(supercell) 

write_lammps_data('data.CPI', supercell, atom_style = 'full', force_skew=True, units='real')

with open("start.lmp","w") as f:
    print("""
###
dimension       3
boundary        p p p
units           real
atom_style      full

variable        p_id world   1

variable        temperature equal 300.0
variable        temperature2 equal 300.0
variable        tempDamp equal 100 # ps

variable        pressure equal 1.00
variable        pressureDamp equal 500 #ps

variable        seed world 1428

read_data       data.CPI

mass            1 132.904999 # Cs
mass            2 126.90447  # I
mass            3 204.199997 # Pb

variable        freq equal 500


pair_style      lj/cut/coul/long 10.0 10.0
pair_modify     mix arithmetic
neighbor        0.3 bin
neigh_modify    check yes delay 0
kspace_style    pppm 1e-4
dielectric 1.0

# original
#variable ACsI   equal "0.0467571702"
#variable rCsI   equal "4.43790"

# original parameters
#variable AI     equal "0.042784417"
#variable rI     equal "5.04975"
#variable ACs    equal "0.0898565009" 
#variable rCs    equal "3.521013"
#variable ACsI   equal "14.0467571702"
#variable rCsI   equal "3.39790"
#variable APbI   equal "4.7050"
#variable rPbI   equal "2.800"
#variable APbCs   equal "4.01050"
#variable rPbCs   equal "4.100"

variable APb     equal "0.0051886219"
variable rPb     equal "4.043"
variable AI      equal "0.042784417"
variable rI      equal "5.04975"
variable ACs     equal "0.089856500"
variable rCs     equal "3.521013"

variable ACsI    equal "0.046757170"
variable rCsI    equal "4.40408"
variable APbI    equal "0.507295"
variable rPbI    equal "3.103"


pair_coeff      1 1 ${ACs}    ${rCs} 
pair_coeff      2 2 ${AI}     ${rI}  
pair_coeff      3 3 ${APb}    ${rPb}     
pair_coeff      1 2 ${ACsI}   ${rCsI}    
pair_coeff      2 3 ${APbI}   ${rPbI}     


thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz
restart         ${freq} restart.0 restart.2

# Minimization
min_style cg
fix 1 all box/relax aniso 0.0 vmax 0.01
minimize        1.0e-3 1.0e-5 1000 10000
unfix 1
write_data      data.min
reset_timestep  0

# NVT
dump            myDump1 all atom 500 out.0.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature2} ${tempDamp} ${seed}
fix             2 all nve

timestep        2.0

velocity        all create ${temperature} ${seed} dist gaussian
run             10000

unfix           1
unfix           2

write_data      data.NVT

undump          myDump1
reset_timestep  0

# NPT

dump            myDump2 all atom 500 out.hexagonal.lammpstrj 

fix             1 all temp/csvr ${temperature2} ${temperature2} ${tempDamp} ${seed}
fix             2 all nph tri ${pressure} ${pressure} ${pressureDamp} 
fix             3 all momentum 10000 linear 1 1 1

run             10000

unfix           1
unfix           2
unfix           3

undump          myDump2

reset_timestep  0

write_restart   restart.file
write_data      data.eq

""",file=f)


Atoms(symbols='Cs32I96Pb32', pbc=True, cell=[20.8684, 19.162, 17.761], initial_charges=..., spacegroup_kinds=...)


In [26]:
subprocess.run("nohup lmp_intel_cpu_intelmpi < start.lmp &",shell=True)

CompletedProcess(args='nohup lmp_intel_cpu_intelmpi < start.lmp &', returncode=0)

LAMMPS (23 Jun 2022)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (20.8684 19.162 17.761) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  160 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.004 seconds
PPPM initialization ...
  using 12-bit tables for long-range coulomb (../kspace.cpp:342)
  G vector (1/distance) = 0.28349713
  grid = 15 12 12
  stencil order = 5
  estimated absolute RMS force accuracy = 0.015786741
  estimated relative force accuracy = 4.7541301e-05
  using single precision MKL FFT
  3d grid and FFT values/proc = 5780 2160
Generat

      9000   298.67803     -14214.408      141.55809     -14072.85       17617.005      19.604644      18.68374       16.34012       0              0              0            
      9500   259.86914     -14229.409      123.16467     -14106.244      17658.647      19.604644      18.68374       16.34012       0              0              0            
     10000   298.29604     -14213.332      141.37705     -14071.955      16948.28       19.604644      18.68374       16.34012       0              0              0            
Loop time of 4.46629 on 1 procs for 10000 steps with 160 atoms

Performance: 386.898 ns/day, 0.062 hours/ns, 2238.993 timesteps/s
100.0% CPU use with 1 MPI tasks x 1 OpenMP threads

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 2.0136     | 2.0136     | 2.0136     |   0.0 | 45.08
Bond    | 0.0009674  | 0.0009674  | 0.0009674  |   0.0 |  0.02
Kspace